# 9.4.2022 Risk Match Stats Breakdown (v0.4)

The following data is from the Risk game played on **September 4, 2022** with Omar, Harsimar, Steven, Renzo, and Josh. If you're interested in the statistics of the game itself, ignore the "Required Functions" and "Cleaning Dataset" header. The notebook was written by Joshua Vargas.

## Changes
v0.2
- v0.2 is intended to test adding the new column, "Winning Side", to help with counting sides that won. DONE!
- Continue building through this version. Archive v1 DONE!

v0.3
- Combine the winCount() and lossCount() functions into one function. DONE!
- Rename and combine combinedTotalCounts() and combinedIndividualCounts() functions. DONE!
- Design function to display all the rolls per side in the displayRolls() function. DONE!
- [eventually move all functions to the same location.]

v0.4
- Start cleaning up and reorganizing functions, etc. 
- Revise docstrings and comments.

In [1]:
# Library imports.
import numpy as np
import pandas as pd
from tabulate import tabulate

In [2]:
# Importing stats sheet.
riskGame = pd.read_csv("Risk Game Statistics 9_4_2022 v0.4.csv")
playerList = list(riskGame["Attacking Player"].unique())

# Required Functions

The section defines all the necessary function for setting up and displaying the information. If you are interested in the statistics, **this section can be skipped.** \
Function list: 
- rollSplit(df,entry) [Used in "Cleaning Dataset" section.]
- winLossCount(df,player,winLoss) [Used in "Player Battle Win/Loss"]
- assortedData(df,players,assortedType) [Used in "Player Battle Win/Loss"]
- playerRollCount(df,player) [Used in "Roll Data"]
- displayRolls(df,players,side) [Used in "Roll Data"]

In [5]:
# Function that splits apart the data and removes the unsplit entry.
def rollSplit(df,entry):
    """
    rollSplit() splits turns with two different rolls into separate
    columns for counting. The function outputs a DataFrame.
    Parameters:
    - df - DataFrame; The DataFrame with info to split.
    - entry - int; The DataFrame entry to check for a
      split.
    """
    
    if type(df["Defending Player"][entry]) == float:
        return df
    # Ignores the entry if the roll is a np.nan.
    # If there was no roll value, the DataFrame remains the same.
    
    elif len(str(df["Highest Attack Roll(s)"][entry])) == 3:
        df.loc[len(df)+entry] = [np.nan,
                               df["Attacking Player"][entry],
                               df["Defending Player"][entry],
                               df["Highest Attack Roll(s)"][entry][0],
                               df["Highest Defend Roll(s)"][entry][0],
                               df["Winning Side"][entry]]
        # Adding columns of rolls to the left of comma
    
        df.loc[len(df)+entry] = [np.nan,
                                 df["Attacking Player"][entry],
                                 df["Defending Player"][entry],
                                 df["Highest Attack Roll(s)"][entry][2],
                                 df["Highest Defend Roll(s)"][entry][2],
                                 np.nan]
        # Adding column of rolls to right of comma
    
        df = df.drop(labels=entry,axis=0)
        # Removing that entry
    
        return df
    # If there are 3 characters in the string (i.e. 2,1), runs
    # process of splitting information. 
    
    else: 
        return df
    # IF there's any other case, the same DataFrame is maintained.

In [4]:
# Function to tally win or loss count per player.
def winLossCount(df,player,winLoss):
    """
    winLossCount() counts the amount of times a player wins or 
    loses a battle on attack or defense. The output returns
    a dictionary that outlines the wins or losses of each side.
    Parameters: 
    df - DataFrame; The data of concern.
    player - string; The player to tally wins and losses.
    winLoss - string; Two choices, "Wins", "Losses"
    """
    
    winLossDict = {}
    # Empty dictionary to be built up by later blocks.
    
    attackPlayerMask = df["Attacking Player"] == player
    defendPlayerMask = df["Defending Player"] == player
    # Creates mask for given player.
    
    if winLoss == "Wins":
        winningAttackMask = df["Winning Side"] == "A"
        winningDefendMask = riskGameCopy["Winning Side"] == "D"
        # Creates mask for each time the player won on attack
        # and defense. 
        
        attackWinCount = len(df[winningAttackMask & attackPlayerMask]["Winning Side"])
        defendWinCount = len(riskGameCopy[winningDefendMask & defendPlayerMask]["Winning Side"])
        totalWinCount = attackWinCount + defendWinCount
        # Counts the amount of times the player won on attack
        # and defense.
        
        winLossDict["Attack "+winLoss] = attackWinCount
        winLossDict["Defend "+winLoss] = defendWinCount
        winLossDict["Total "+winLoss] = totalWinCount
        # Adds the elements to the dictionary to track data.
    
    elif winLoss == "Losses":
        losingAttackMask = df["Winning Side"] == "D"
        losingDefendMask = riskGameCopy["Winning Side"] == "A"
        # Creates mask for each time the player lost on attack
        # and defense. 
        
        attackLossCount = len(df[losingAttackMask & attackPlayerMask]["Winning Side"])
        defendLossCount = len(riskGameCopy[losingDefendMask & defendPlayerMask]["Winning Side"])
        totalLossCount = attackLossCount + defendLossCount
        # Counts the amount of times the player lost on attack
        # and defense
        
        winLossDict["Attack "+winLoss] = attackLossCount
        winLossDict["Defend "+winLoss] = defendLossCount
        winLossDict["Total "+winLoss] = totalLossCount
        # Adds the elements to the dictionary to track data.
    
    else:
        raise TypeError("Variable \"winLoss\" must be either \"Wins\" or \"Losses\".")
    
    return winLossDict

In [5]:
# Function to create lists of the data.
def assortedData(df,players,assortedType):
    """
    assortedData() creates a list of lists to sort the
    data to work with the tabular function. The function
    outputs a list of lists.
    Parameters:
    - df - DataFrame; DataFrame of information to use. 
    - players - list; List of all players in the game. 
    - assortedType - string; Two inputs, "totalWinLoss",
      "sidedWinLoss".
    """
    
    combinedList = []
    # Empty list to add all the data later.
    
    if assortedType == "totalWinLoss":
        for player in players:
            playerWinLoss = [player,
                             winLossCount(df,player,"Wins")["Total Wins"],
                             winLossCount(df,player,"Losses")["Total Losses"],
                             winLossCount(df,player,"Wins")["Total Wins"]-winLossCount(df,player,"Losses")["Total Losses"]]
            
            # Creates a list of the information regarding total
            # wins and losses for every player.
        
            combinedList.append(playerWinLoss)
            # Adds information to the blank list.
    
    elif assortedType == "sidedWinLoss":
        for player in players:
            playerWinLoss = [player,
                             winLossCount(df,player,"Wins")["Attack Wins"],
                             winLossCount(df,player,"Wins")["Defend Wins"],
                             winLossCount(df,player,"Losses")["Attack Losses"],
                             winLossCount(df,player,"Losses")["Defend Losses"]]
            # Creates a list of the information regarding sided
            # wins and losses for every player.
        
            combinedList.append(playerWinLoss)
            # Adds information to the blank list. 
    
    else:
        errorMsg = "Variable \"assortedType\" must be either \"totalWinLoss\" or \"sidedWinLoss\"."
        raise TypeError(errorMsg)
        
    return combinedList

In [6]:
# Function to count the number of rolls per player.
def playerRollCount(df,player):
    """
    playerRollCount() counts the amount of times a specified player 
    rolled each number. The roll splitter counts per-side rolls and
    total rolls. 
    Parameters:
    - df - DataFrame; The data to process and count rolls.
    - player - string; The player to tally up information.
    """
    
    attackPlayerMask = df["Attacking Player"] == player
    defendPlayerMask = df["Defending Player"] == player
    # Creates masks for given player
    
    rollList = ["1","2","3","4","5","6"]
    attackRollCount = []
    defendRollCount = []
    playerTally = {}
    # Creating list/numpy arrays that holds the number of rolls
    
    for roll in rollList:
        attackRollMask = df["Highest Attack Roll(s)"] == rollList[int(roll)-1]
        attackRollEntry = len(df[attackPlayerMask & attackRollMask])
        attackRollCount.append(attackRollEntry)
        playerTally["Attack " + roll] = attackRollEntry
        # Creates the list and adds the dictionary entry of the player's
        # attack side rolls.
        
        defendRollMask = df["Highest Defend Roll(s)"] == rollList[int(roll)-1]
        defendRollEntry = len(df[defendPlayerMask & defendRollMask])
        defendRollCount.append(defendRollEntry)
        playerTally["Defend " + roll] = defendRollEntry
        # Creates the list and adds the dictionary entry of the player's
        # defend side rolls.
    
    return playerTally

In [7]:
# Function to format roll data in a table format.
def displayRolls(df,players,side):
    """
    displayRolls() creates a format for the tabular function to display 
    all the rolls based on which side was selected. The output is a list 
    of lists. 
    Parameters:
    - df - DataFrame; The data to process into displaying their rolls.
    - players - list; List of all the players in the game. 
    - side - string; A string of two possible outputs, "Attack" or
      "Defend".
    """
    
    displayList = []
    
    if side == "Attack":
        for player in players:
            playerSubset = [player]
            # Creates a list to add each attack element. 
            
            for rollNumber in range(6):
                rollNumberCount = playerRollCount(df,player)["Attack "+str(rollNumber+1)]
                playerSubset.append(rollNumberCount)
            # Takes each attack roll number from the dictionary
            # and adds it to the "playerSubset" list.
            
            displayList.append(playerSubset)
            # Adds the "playerSubset" list to the displayList to
            # create one list for all players.
        
    elif side == "Defend":
        for player in players:
            playerSubset = [player]
            # Creates a list to add each attack element. 
            
            for rollNumber in range(6):
                rollNumberCount = playerRollCount(df,player)["Defend "+str(rollNumber+1)]
                playerSubset.append(rollNumberCount)
            # Takes each attack roll number from the dictionary
            # and adds it to the "playerSubset" list.
            
            displayList.append(playerSubset)
            # Adds the "playerSubset" list to the displayList to
            # create one list for all players.
        
    else:
        errorMsg = "Variable \"side\" must be either \"Attack\" or \"Defend\"."
        raise TypeError(errorMsg)
    
    return displayList

# Cleaning Dataset

This section includes code for setting up the data to be displayed later. Be sure to only run this section once, unless you re-run the riskGame DataFrame. If you are interested in the statistics, **this section can be skipped.** \
Written functions used:
- rollSplit(df,entry)

In [6]:
# Splitting sections with two rolls into individual ones.
riskGameCopy=riskGame

for i in list(riskGameCopy.index):
    riskGameCopy=rollSplit(riskGameCopy,i)
# Splits all entries with two rolls into their own cells.

# Game Summary

The overview of the statistics of the game. These statistics were collected from manually tallying from the game and tracking on the spreadsheet. The categories tracked are "Starting Territory", "Trade Ins", "Territory Count", "Final Score". 


"Territory Count" refers to the total amount of territories a player had at the end of the game, whenever it was tracked. \
"Final Score" is tallied by summing the total amount of territories a player has plus adding extra points given by controlling a territory. For example, if a player has 9 territories and controls Australia, their score adds an extra 4 points for each territory in the continent they control.


This section is hard-coded from the tracking done on the spreadsheet. This must be manually input for each game.

In [9]:
# Displaying general statistics.
genStats = [
    ["Harsimar",9,2,8,12],
    ["Renzo",8,2,7,11],
    ["Josh",8,2,11,11],
    ["Omar",9,2,10,10],
    ["Steven",8,1,5,5]
]

genHeaders = ["Starting Territory","Trade Ins","Territory Count","Final Score"]

print(tabulate(genStats,headers=genHeaders,tablefmt="fancy_grid"))

╒══════════╤══════════════════════╤═════════════╤═══════════════════╤═══════════════╕
│          │   Starting Territory │   Trade Ins │   Territory Count │   Final Score │
╞══════════╪══════════════════════╪═════════════╪═══════════════════╪═══════════════╡
│ Harsimar │                    9 │           2 │                 8 │            12 │
├──────────┼──────────────────────┼─────────────┼───────────────────┼───────────────┤
│ Renzo    │                    8 │           2 │                 7 │            11 │
├──────────┼──────────────────────┼─────────────┼───────────────────┼───────────────┤
│ Josh     │                    8 │           2 │                11 │            11 │
├──────────┼──────────────────────┼─────────────┼───────────────────┼───────────────┤
│ Omar     │                    9 │           2 │                10 │            10 │
├──────────┼──────────────────────┼─────────────┼───────────────────┼───────────────┤
│ Steven   │                    8 │           1 │     

# Player Battle Win/Loss

This section displays each player's total battle count and their win/loss ratio. The win/loss is broken up into attack wins and defend wins. \
Written functions used:
- winLossCount(df,player,winLoss)
- assortedData(df,players,assortedType)

In [11]:
# Displaying win/loss tables.
colNames1 = ["Player(s)",
            "Win Count","Loss Count","Win Loss"]

colNames2 = ["Player(s)","Attack Win","Defend Win",
            "Attack Loss","Defend Loss"]

# Table 1, Total Win/Loss
print("Total Win/Loss:")
print(tabulate(assortedData(riskGameCopy,playerList,"totalWinLoss"),
                   headers=colNames1,
                   tablefmt="fancy_grid"))

# Table 2, Sided Win/Loss
print("Sided Win/Loss:")
print(tabulate(assortedData(riskGameCopy,playerList,"sidedWinLoss"),
                   headers=colNames2,
                   tablefmt="fancy_grid"))

Total Win/Loss:
╒═════════════╤═════════════╤══════════════╤════════════╕
│ Player(s)   │   Win Count │   Loss Count │   Win Loss │
╞═════════════╪═════════════╪══════════════╪════════════╡
│ Omar        │          14 │           11 │          3 │
├─────────────┼─────────────┼──────────────┼────────────┤
│ Harsimar    │          14 │           11 │          3 │
├─────────────┼─────────────┼──────────────┼────────────┤
│ Steven      │          10 │           13 │         -3 │
├─────────────┼─────────────┼──────────────┼────────────┤
│ Renzo       │          10 │           14 │         -4 │
├─────────────┼─────────────┼──────────────┼────────────┤
│ Josh        │          11 │           10 │          1 │
╘═════════════╧═════════════╧══════════════╧════════════╛
Sided Win/Loss:
╒═════════════╤══════════════╤══════════════╤═══════════════╤═══════════════╕
│ Player(s)   │   Attack Win │   Defend Win │   Attack Loss │   Defend Loss │
╞═════════════╪══════════════╪══════════════╪═════════════

# Roll Data

This section compiles all the rolls of every player on attack and defense and sorts them into respective tables. The sum of their rolls are also combined in another table. \
Written functions used:
- playerRollCount(df,player) 
- displayRolls(df,players,side)

In [12]:
# Generating tables for rolls per side per player.
colRollDisplay = ["Player",
                  "1","2","3",
                  "4","5","6"]

# Table 1, Attack Side
print("Attack Side:")
print(tabulate(displayRolls(riskGameCopy,playerList,"Attack"),
              headers=colRollDisplay,
              tablefmt="fancy_grid"))

# Table 2, Defense Side
print("Defense Side:")
print(tabulate(displayRolls(riskGameCopy,playerList,"Defend"),
              headers=colRollDisplay,
              tablefmt="fancy_grid"))

Attack Side:
╒══════════╤═════╤═════╤═════╤═════╤═════╤═════╕
│ Player   │   1 │   2 │   3 │   4 │   5 │   6 │
╞══════════╪═════╪═════╪═════╪═════╪═════╪═════╡
│ Omar     │   2 │   4 │   9 │  15 │  14 │  15 │
├──────────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ Harsimar │   1 │   6 │   7 │  10 │   9 │  12 │
├──────────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ Steven   │   3 │   4 │   4 │   7 │   5 │  11 │
├──────────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ Renzo    │   3 │   6 │   7 │  19 │  11 │  13 │
├──────────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ Josh     │   2 │   4 │   4 │   6 │   7 │   9 │
╘══════════╧═════╧═════╧═════╧═════╧═════╧═════╛
Defense Side:
╒══════════╤═════╤═════╤═════╤═════╤═════╤═════╕
│ Player   │   1 │   2 │   3 │   4 │   5 │   6 │
╞══════════╪═════╪═════╪═════╪═════╪═════╪═════╡
│ Omar     │   4 │  14 │  10 │   7 │   7 │   5 │
├──────────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ Harsimar │  10 │   7 │   3 │   6 │  12 │  12 │
├──────────┼─────┼─────┼─────┼─────┼─────┼